In [1]:
from pysndfx import AudioEffectsChain
import wubwub as wb
import wubwub.sounds as snd

ORGAN = snd.load('keys.organ')
DRUMS = snd.load('drums.house')
DRUMS2 = snd.load('drums.ukhard')
FX = snd.load('synth.fx')
SYNTH = snd.load('synth.ts_synth')

end = 40
seq = wb.Sequencer(bpm=400, beats=40)

emph = wb.Pattern([1, 4, 7, 9], length=10).until(end)
notemph = wb.Pattern([2, 3, 5, 6, 8, 10], length=10).until(end)
every = emph.merge(notemph)
kickpat = wb.Pattern([1, 4, 6], length=10).until(end)
snarepat = wb.Pattern([7, 10], length=10).until(end)
phase = wb.Pattern([1], length=4).until(end)
bleeppat = wb.Pattern([1,2,], length=4).until(end)
synthpat = wb.Pattern([1, 4, 6, 7], length=10).until(end)
count = wb.Pattern([1, 3, 5, 7, 9], length=10).until(end)

organ = seq.add_sampler(ORGAN['C1'], name='organ', basepitch='C1')
organ.make_notes(every, pitches=[3,2,0,5,3,2,7,3,5,2])

hat = seq.add_sampler(DRUMS['hat1'], name='hat')
hat.make_notes(emph, volumes=0)
hat.make_notes(notemph, volumes=-5, lengths=.1)
hat.volume -= 10
hat.pan = .5

snare = seq.add_sampler(DRUMS['snare3'], name='snare')
snare.make_notes(snarepat, volumes=[0, -5])
snare.volume -= 5

kick = seq.add_sampler(DRUMS['kick6'], name='kick')
kick.make_notes(kickpat)
kick.volume -= 2
organ[kick.array_of_beats()] = wb.alter_notes(organ[kick.array_of_beats()], volume=-5)

ride = seq.add_sampler(DRUMS2['ride-hard'], name='ride')
ride.make_notes(phase, lengths=5)
ride.pan = -.6
ride.effects = AudioEffectsChain().highpass(5000)

bleep = seq.add_sampler(wb.shift_pitch(FX['checkpoint-hit'], 8), name='bleep')
bleep.make_notes(bleeppat)
bleep.effects = AudioEffectsChain().reverb(wet_gain=5)
bleep.volume -= 10

synth = seq.add_sampler(SYNTH['patch001'], name='synth')
synth.make_notes(every, pitches=[12,12,12,0,0,0,12,12,0,0], lengths=.2)
synth.effects = AudioEffectsChain().highpass(1000).reverb()

intro = seq.copy(with_notes=False)
intro['organ'].add_fromdict(seq['organ'].notedict)
intro['organ'].effects = AudioEffectsChain().lowpass(200)

intro['hat'].make_notes(count.onmeasure(3, measurelen=10), lengths=.1)

intro['snare'].make_notes([37, 38, 39, 40])

final = wb.join([intro, seq])
final.build(overhang=5)